In [13]:
import torch
from dereverb_net.model import ConvTas_RIR_net  # Assuming this is your model class

# Initialize model
model = ConvTas_RIR_net()  # Adjust parameters as needed
device = torch.device("cpu")
    
# Load checkpoint
checkpoint_path = "checkpoint_epoch_5.pth"
checkpoint = torch.load(checkpoint_path, map_location=device)
    
# Load model state dict
model.load_state_dict(checkpoint['model_state_dict'])
model = model.to(device)
    
# Set model to evaluation mode
model.eval()


ConvTas_RIR_net(
  (encoder): Encoder(
    (filterbank): FreeFB()
  )
  (decoder): Decoder(
    (filterbank): FreeFB()
  )
  (masker): TDConvNet(
    (bottleneck): Sequential(
      (0): GlobLN()
      (1): Conv1d(512, 128, kernel_size=(1,), stride=(1,))
    )
    (TCN): ModuleList(
      (0): Conv1DBlock(
        (shared_block): Sequential(
          (0): Conv1d(128, 512, kernel_size=(1,), stride=(1,))
          (1): PReLU(num_parameters=1)
          (2): GlobLN()
          (3): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(1,), groups=512)
          (4): PReLU(num_parameters=1)
          (5): GlobLN()
        )
        (res_conv): Conv1d(512, 128, kernel_size=(1,), stride=(1,))
        (skip_conv): Conv1d(512, 128, kernel_size=(1,), stride=(1,))
      )
      (1): Conv1DBlock(
        (shared_block): Sequential(
          (0): Conv1d(128, 512, kernel_size=(1,), stride=(1,))
          (1): PReLU(num_parameters=1)
          (2): GlobLN()
          (3): Conv1d(512, 512, kerne

In [14]:
import torchaudio
from IPython.display import Audio, display

rs_file = "data/synthetic/test-real-rir/121_123859_000017_000001_00252.wav"

wav, sr = torchaudio.load(rs_file)
wav = wav[:, :32000]
wav = wav.to(device)

clean_wav, rir_wav = model(wav)
print(clean_wav.shape)


torch.Size([1, 1, 32000])


In [15]:
# check loss

from dereverb_net.loss_fn import wav_loss

loss = wav_loss()
loss = loss(wav, clean_wav)
print(loss)

## SEEMS THAT THE MULTI-SCALE LOSS IS NOT WORKING
### OPTIMIZING MULTI-SCALE LOSS LEADS TO BAD RESULTS

(tensor(0.2165, grad_fn=<MulBackward0>), tensor(1.2402e+08, grad_fn=<MulBackward0>))
